<a href="https://colab.research.google.com/github/sagar9926/MTech_Atificial_Intelligence/blob/main/ML1/ML_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1 

## Dataset : Iris Dataset

Use sklearn library for loading iris dataset.

__Aim__: Classification using Naive Bayes classifier

```
● Apply Naive bayes classifier assuming all features are independent.
Do not use any predefined library for classification
Report overall accuracy, class wise accuracy, confusion matrix and ROC curve.
```

In [1]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt

### Loading Dataset

In [2]:
iris = datasets.load_iris()
df_iris = pd.DataFrame(iris.data,columns = iris.feature_names)
df_iris['target'] = iris.target

In [3]:
X, y = df_iris.iloc[:, :-1], df_iris.iloc[:, -1]

# # split on train and test 0.7/0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=1, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(75, 4) (75,)
(75, 4) (75,)


In [4]:
df_iris.target.unique()

array([0, 1, 2])

### Creating a Naive Bayes class

In [12]:
class NaiveBayesClassifier:

  """
  Bayes Theorem : 

  P(y|X) = (P(X|y)*P(y))/P(X)

  where :
  X : Input Features
  y : Traget Variable
  P(y|X) = Posterior Probability
  P(X|y) = Liklihood
  P(y) = Prior
  P(X) = Evidence

  """

  def __init__(self,prior = None):

    self.liklihood = {}
    self.posteriors = []
    self.prediction = []
    self.mean = None
    self.variance = None 
    if prior :
      self.prior = prior
    else :
      self.prior = {} 
 
  def calculate_prior(self ,target ,classes):
    """
    This function calculates the prior probabilities
    """
    prior = {}
    for index in classes:
      prior[index] = (target == index).sum()/len(target) 
    return prior


  def feature_statistics(self , features , target):
    """
    This function calculates mean and variance for continuous features
    """
    var = lambda x : np.var(x)  
    mean = features.groupby(target).mean().reset_index()
    #mean = features.groupby(y_train).agg({'sepal length (cm)' : 'mean',	'sepal width (cm)': 'mean',	'petal length (cm)': 'mean',	'petal width (cm)': 'mean'}).reset_index()
    #variance = features.groupby(y_train).agg({'sepal length (cm)' : var,	'sepal width (cm)': var,	'petal length (cm)': var,	'petal width (cm)': var}).reset_index()
    variance = features.groupby(target).var(ddof = 0).reset_index()
    return mean , variance


  def gaussian_probability(self,test_feature,classes, mean, variance):
    liklihood_prob = {}
    for index in classes:
      mean_vector = np.array(mean[mean['target'] == index])[0][1:]
      variance_vector = np.array(variance[variance['target'] == index])[0][1:]
      mean_diff_square = (np.array(test_feature) - mean_vector)**2
      liklihood_prob[index] = (1/np.sqrt(2*np.pi*variance_vector)*np.exp(-1*0.5*mean_diff_square/variance_vector))
    return liklihood_prob


  def calculate_posterior(self,test_feature,classes):

    posteriors = {}
    self.liklihood = self.gaussian_probability(test_feature ,classes ,self.mean,self.variance)
    for index in classes :
      posteriors[index] = reduce(lambda x , y : x*y , self.liklihood[index]) * self.prior[index]
    return posteriors


  def fit(self, features, target):
    self.classes = target.unique()
    if self.prior :
      pass
    else:
      self.prior = self.calculate_prior(target , self.classes)
    self.mean , self.variance = self.feature_statistics(features , target)
    

  def predict(self,test_features):
    for i in range(len(test_features)):
      self.posteriors.append(self.calculate_posterior(test_features.iloc[i].values,self.classes))
      self.prediction.append(sorted(self.posteriors[i].items(),key = lambda x : x[1],reverse = True)[0][0])
    #return(self.prediction)

  def accuracy(self , target,names):

    overall_accuracy = sum(target == self.prediction) / len(target)
    print(f"Overall accuracy of data : {round(overall_accuracy*100,2)}%")

    data = pd.DataFrame({'Actual Class' : target.values,'Predicted Class':self.prediction})
    for class_ in target.unique():
      temp = data[data['Actual Class'] == class_]
      print(f"{names[class_]} accuracy of data : {round(sum(temp['Actual Class'] == temp['Predicted Class']) / len(temp)*100,2)}%")

  def confusion_matrix(self,target):
    df_temp = pd.DataFrame({'Actual Class' : target,'Predicted Class' : self.prediction})
    Confusion_matrix = df_temp.groupby(['Predicted Class','Actual Class']).agg({'Predicted Class':'count'})
    Confusion_matrix.columns = ['Instance_count']
    Confusion_matrix['Instance_count'] = Confusion_matrix['Instance_count'].astype(int)
    Confusion_matrix.reset_index(inplace = True)
    Confusion_matrix = Confusion_matrix.pivot(index='Actual Class', columns='Predicted Class', values='Instance_count').fillna(0)
    return Confusion_matrix



### Model Training

In [13]:
model = NaiveBayesClassifier()
model.fit(X_train, y_train)

In [14]:
model.prior

{0: 0.3333333333333333, 1: 0.3333333333333333, 2: 0.3333333333333333}

### Making predictions using trained model

In [15]:
model.predict(X_test) 

In [16]:
model.accuracy(y_test,iris.target_names)

Overall accuracy of data : 97.33%
virginica accuracy of data : 96.0%
setosa accuracy of data : 100.0%
versicolor accuracy of data : 96.0%


In [17]:
model.confusion_matrix(y_test)

Predicted Class,0,1,2
Actual Class,,,
0,25.0,0.0,0.0
1,0.0,24.0,1.0
2,0.0,1.0,24.0


# Question 2 :

__Dataset__: Wine dataset (use sklearn library for loading the dataset)

__Aim__: Naive Bayes Classification
```
Shuffle the data with seed value 42 and perform a 70- 30 stratified split of the data into a train and test set.
Also, plot the class-wise distribution of data in the train and test set (one for train set and one for test set).

Compare the distributions. Now, perform classification as follows:

* Train a Gaussian Naive Bayes classifier and report (a) the class priors, (b) mean and variance of
each feature per class.

* Train another Gaussian Naive Bayes classifier by setting prior probability for the classes. Repeat this
experiment by setting priors in the ratios: (a) 40-40-20 and (b) 80-10-10.
```

In [18]:
wine = datasets.load_wine()
df_wine = pd.DataFrame(wine.data,columns = wine.feature_names)
df_wine['target'] = wine.target

In [19]:
X, y = df_wine.iloc[:, :-1], df_wine.iloc[:, -1]

# # split on train and test 0.7/0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(124, 13) (124,)
(54, 13) (54,)


In [20]:
df_wine.target.unique()

array([0, 1, 2])

### Plotting Distributions

In [21]:
df_wine

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


### Model Training

In [22]:
model = NaiveBayesClassifier()
model.fit(X_train, y_train)

### Making predictions using trained model

In [23]:
model.predict(X_test) 

In [24]:
model.accuracy(y_test,wine.target_names)

Overall accuracy of data : 100.0%
class_0 accuracy of data : 100.0%
class_1 accuracy of data : 100.0%
class_2 accuracy of data : 100.0%


In [25]:
model.prior

{0: 0.33064516129032256, 1: 0.4032258064516129, 2: 0.2661290322580645}

In [26]:
model.confusion_matrix(y_test)

Predicted Class,0,1,2
Actual Class,,,
0,18.0,0.0,0.0
1,0.0,21.0,0.0
2,0.0,0.0,15.0


#### Adjusting prior to 40-40-20

In [36]:
prior = {0: 0.40, 1: 0.40, 2: 0.20}
model = NaiveBayesClassifier(prior)
model.fit(X_train, y_train)
model.predict(X_test) 
model.accuracy(y_test,wine.target_names)

Overall accuracy of data : 98.15%
class_0 accuracy of data : 100.0%
class_1 accuracy of data : 95.24%
class_2 accuracy of data : 100.0%


In [33]:
model.confusion_matrix(y_test)

Predicted Class,0,1,2
Actual Class,,,
0,18.0,0.0,0.0
1,1.0,20.0,0.0
2,0.0,0.0,15.0


#### Adjusting prior to 80-10-10

In [38]:
prior = {0: 0.80, 1: 0.10, 2: 0.10}
model = NaiveBayesClassifier(prior)
model.fit(X_train, y_train)
model.predict(X_test) 
model.accuracy(y_test,wine.target_names)

Overall accuracy of data : 98.15%
class_0 accuracy of data : 100.0%
class_1 accuracy of data : 95.24%
class_2 accuracy of data : 100.0%


In [39]:
model.confusion_matrix(y_test)

Predicted Class,0,1,2
Actual Class,,,
0,18.0,0.0,0.0
1,1.0,20.0,0.0
2,0.0,0.0,15.0
